In [ ]:
import os

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [ ]:
from lib import *

In [ ]:
import glob
from dateutil import tz
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
def evaluateNash(data_obs, data_sim, round_el=5):
    from numpy import nanmean, nansum
    num_nse = nansum((data_obs - data_sim)**2)
    den_nse = nansum((data_obs - nanmean(data_obs))**2)
    nse = 1 - num_nse/den_nse
    return round(nse, round_el)

In [ ]:
def evaluateNNSE(nse, round_el=5):
    return round(1/(2-nse), round_el)

In [ ]:
def plot_current(forecast_df, observed_df, filename, output_path):
    
    plots = []

    ### fct plot ###
    plt_conf = {}
    plt_conf["label"] = 'forecast'
    plt_conf["color"] = '#5e3c99'
    plots.append( (forecast_df, plt_conf) )

    ### obs plot ###
    plt_conf = {}
    plt_conf["label"] = 'Observed'
    plt_conf["color"] = '#fdb863'
    plots.append( (observed_df, plt_conf) )

    createPlot(plots,  "Time $[hour]$", "Streamflow $[m^3/hour]$",
           output_path + "HD/" + filename + "." + output_format, output_format=output_format, my_dpi=600)

    createPlot(plots,  "Time $[hour]$", "Streamflow $[m^3/hour]$",
           output_path + "LD/"+ filename + "." + output_format, output_format=output_format, my_dpi=50)

In [ ]:
wdir = "/media/windows/projects/icon-evaluation/hydro_modeling/"
os.chdir(wdir)

basin = "passirio"

start_date_str = '2021-07-01 00:00:00'
end_date_str = '2021-10-01 00:00:00'
timezone_str = 'Europe/Rome'

start_date = dt.datetime.strptime(
    start_date_str, '%Y-%m-%d %H:%M:%S')
end_date = dt.datetime.strptime(
    end_date_str, '%Y-%m-%d %H:%M:%S')

dates = [start_date + dt.timedelta(hours=x)
         for x in range(1, (end_date-start_date).days*24 + 1)]

# station_meta = [{"point": "15",
#                  "names": "plan",
#                  "gauge": "155"}]

station_meta = [{"point": "58",
                 "names": "merano",
                 "gauge": "118"}]
                 
gauge_station = '58' ### merano
gauge_measure = '118'

# gauge_station = '15' ### plan
# gauge_measure = '155'

flow_min = 0
flow_max = 200

nash_months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [ ]:
model_path =  wdir + basin + "/"
output_path =  wdir + basin + "/results/"
mkNestedDir(output_path)

In [ ]:
obs_path = model_path + "meteo/streamflow/"
stations_path = glob.glob(obs_path + "*.txt")

obs_dataframe = pd.DataFrame(index=dates)
obs_dataframe.index.name = 'datetime'

for obs_flow_filepath in stations_path:

    station_name = os.path.basename(obs_flow_filepath)[:-4]

    obs_flow = pd.read_csv( obs_flow_filepath, skiprows=5, index_col=0, names=[station_name])
    obs_flow.index.name = 'datetime'
    obs_flow.index = [dt.datetime.strptime(
        i, '%Y-%m-%d %H:%M:%S') for i in obs_flow.index]

    obs_flow = obs_flow[ obs_flow[station_name] >= flow_min ]
    obs_flow = obs_flow[ obs_flow[station_name] <= flow_max ]

    obs_dataframe[station_name] = obs_flow

obs_dataframe_partial = obs_dataframe[obs_dataframe.index.month.isin(nash_months)]

In [ ]:
model_dataframe = pd.DataFrame(index=dates)
model_dataframe.index.name = 'datetime'

model_output_path = model_path + "OUTPUT/FORWARD/"
list_output_runs_path = glob.glob(model_output_path + "*/")

nash_table = pd.DataFrame(index=[], columns=['NASH', 'NNSE', 'RMSE', 'MAE'])

for out_dir in list_output_runs_path:

    try:
        run_id = os.path.dirname(out_dir).split("_")[-1]
        nash_table = nash_table.reindex(
            nash_table.index.append(pd.Index([run_id])))

        ## read streamflow from the model - skip IC bug
        model_streamflow = pd.read_csv(
            out_dir + "/out_flow.txt", index_col=0)
        model_streamflow.index.name = 'datetime'
        model_streamflow.index = [dt.datetime.strptime(
            i, '%Y-%m-%d %H:%M') for i in model_streamflow.index]

        for st in station_meta:

            # print(st)

            # model_dataframe[st["point"]] = model_streamflow[model_streamflow[st["point"]].between(flow_min, flow_max)][st["point"]]
            model_dataframe = model_streamflow[[st["point"]]]

            metrics_data = pd.DataFrame()
            metrics_data['obs'] = obs_dataframe_partial[st["gauge"]]
            metrics_data['model'] = model_dataframe[model_dataframe.index.month.isin(nash_months)][st["point"]]
            ### to avoid str in the model results '********' bug of ICHYMOD
            metrics_data['model'] = pd.to_numeric(metrics_data['model'], downcast="float", errors='coerce')

            metrics_data.dropna(how='any', inplace=True)

            # plot_current(metrics_data[['model']], metrics_data[['obs']], str(run_id), output_path)

            try:
                if len(metrics_data) == 0:
                    raise
                nash_table.loc[run_id]['NASH'] = evaluateNash(
                    metrics_data['obs'], metrics_data['model'])
            except:
                print("NASH not evaluated!")
                nash_table.loc[run_id]['NASH'] = np.nan

            try:
                nash_table.loc[run_id]['RMSE'] = mean_squared_error(
                    metrics_data['obs'], metrics_data['model'], squared=True)
            except:
                print("RMSE not evaluated!")
                nash_table.loc[run_id]['RMSE'] = np.nan

            try:
                nash_table.loc[run_id]['MAE'] = mean_absolute_error(
                    metrics_data['obs'], metrics_data['model'])
            except:
                print("MAE not evaluated!")
                nash_table.loc[run_id]['MAE'] = np.nan
            
        #     break
        # break

    except:
        continue

nash_table['NNSE'] = [evaluateNNSE(n) for n in nash_table['NASH']]

In [ ]:
i_max_nash = pd.to_numeric(nash_table['NASH']).idxmax()
max_nash = nash_table['NASH'][i_max_nash]
print( "NSE [-inf:1.0] max: " + str(max_nash) + " at simulation #" + str(i_max_nash) )

20210213 > NSE [-inf:1.0] max: 0.65385 at simulation #216

In [ ]:
nash_table[nash_table['NASH'] > 0.1].sort_values('MAE', ascending=True)

In [ ]:
obs_flow_filepath = wdir + basin + "/OUTPUT/FORWARD/output_{id}/".format(id=i_max_nash)
streamflow_df = pd.DataFrame()

### measured ###
obs_flow = pd.read_csv(wdir + basin + "/meteo/streamflow/" + gauge_measure +
                           ".txt", skiprows=5, header=None, parse_dates=True, index_col=0)
obs_flow.index.name = 'datetime'
obs_flow = obs_flow[start_date:end_date]

streamflow_df['measured'] = obs_flow[obs_flow[1] > 0][1]

### forward ###
model_streamflow = pd.read_csv(obs_flow_filepath + 'out_flow.txt', index_col=0)

model_streamflow = model_streamflow[[st["point"]]]
model_streamflow.index.name = 'datetime'
model_streamflow.index = [dt.datetime.strptime(
            i, '%Y-%m-%d %H:%M') for i in model_streamflow.index]
            
model_streamflow[st["point"]] = pd.to_numeric(model_streamflow[st["point"]], downcast="float", errors='coerce')

model_streamflow = model_streamflow[start_date:end_date]

streamflow_df['forward'] = model_streamflow

streamflow_df.plot(figsize=(15,15))

In [ ]:
precipitation_kr_path = wdir + basin + "/meteo/GS/observations/precipitation.txt"

precipitation_kr = pd.read_csv(precipitation_kr_path, sep="\s+")
precipitation_kr.reset_index(inplace=True)

dates = [ str(date) for date in precipitation_kr['index'] ] 
hours = [ str(hour) for hour in precipitation_kr['date'] ] 

datetimes = []
for i in range(len(dates)):
    d = dt.datetime.strptime(dates[i], '%Y-%m-%d')
    datetimes.append(d + dt.timedelta(hours=int(hours[i])))

precipitation_kr.drop(columns=['index', 'date'], inplace=True)

precipitation_kr.index = datetimes

precipitation_kr = precipitation_kr.mean(axis=1)[start_date:end_date]

precipitation_kr.plot()